# cx_Oracle 모듈
- 파이썬에서 오라클 DBMS와 연동는 다양한 함수를 제공하는 모듈
- Python DB API 2.0 표준을 따른다. 

## 참고
- [cx_Oracle 모듈 홈페이지](https://oracle.github.io/python-cx_Oracle/)
- [Python DB API](https://docs.python.org/ko/3/library/sqlite3.html)

## cx_Oracle 모듈 설치
- [설치 튜토리얼](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)
- 패키지 설치
```
conda install cx_oracle
pip install cx_Oracle --upgrade
```

## 코딩 절차
1. cx_Oracle 모듈 import

2. cx_Oracle모듈의 connect() 함수를 이용해 디비연결 (방법3가지)              
    - 2.1 
    - 2.2 
    - 2.3 

3. Connection 객체의 cursor() 메소드를 이용해 Cursor객체 조회.
    - Cursor객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체

4. Cursor 의 execute() 메소드를 이용해 SQL문 전송

5. SELECT 의 경우 결과 조회.
    - 조회결과 한행씩 반환받아 사용
    - 조회결과 한번에 반환받아 사용

6. INSERT/DELETE/UPDATE의 경우  SQL 문 실행 후 Connection의 commit() 메소드를 이용해 Transaction 처리

7. Cursor, Connection 연결 닫기

### DB 연결
- connect() 함수를 이용하며 연결 후 Connection 객체를 받는다.
- 연결시 필요한 값
    - **host**
    - **port번호**
    - **SID(DB이름)**
    - **user name**
    - **password**

### Connection 주요 메소드
- commit(): 커밋
- rollback(): 롤백
- close(): 연결 닫기

In [3]:
!pip install cx_Oracle

In [2]:
# connect()함수를 이용해서 연결시도 -> 연결 성공 시 Connection 객체를 반환 
# Connection 객체 : 연결정보를 가지고 있는 객체. 
import cx_Oracle 

username = 'c##scott_join'
password = 'tiger'
url = 'localhost:1521/XE' # 형식 : host:post/sid 

conn = cx_Oracle.connect(username, password, url) # 연결시도 -> 성공하면 connection 반환 
print(type(conn)) # 연결 성공하면 <class 'cx_Oracle.Connection'> 반환 
conn.close() # 접속해제 

<class 'cx_Oracle.Connection'>


In [5]:
# username / pwd@url 형태 
conn2 = cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE")
print(type(conn2)) # <class 'cx_Oracle.Connection'>
conn2.close()

<class 'cx_Oracle.Connection'>


#### DSN (Data Source Name) 을 이용한 연결
- DSN: DBMS를 연결하기 위한 정보를 모아 놓은 것. (서로 다른 DBMS의 연결정보를 통일하기 위해서 방식)
- makedsn(host, port번호, SID) 함수 이용해 생성한다.

In [7]:
username = 'c##scott_join'
password = 'tiger'
host = 'localhost'
port = 1521
sid = 'XE'
dsn = cx_Oracle.makedsn(host, port, sid)
# print(dsn)
conn3 = cx_Oracle.connect(username, password, dsn)
print(conn3) # <cx_Oracle.Connection to c##scott_join@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))>
conn3.close()

<cx_Oracle.Connection to c##scott_join@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))>


In [ ]:
# 연결하면 끊어줘야 한다. 이를 자동으로 하기 위해서 with절을 사용해서 편하게 구현 : with block
with cx_Oracle.connect(username, password, dsn) as conn4 : 
# db관련 작업코드 
    pass 

## Cursor 객체 (커서 - 데이터의 현재위치)
- SQL문을 전송하고 select결과를 조회하는 메소드들을 제공한다.
- Connection객체의 cursor() 메소드로 받아온다.

### Cursor의 주요 메소드
#### SQL 실행 메소드
- execute(sql) : 하나의 sql 문 실행. 
- executemany(sql): insert, update, delete 배치 처리
    - insert, update, delete는 다 처리후 conn.commit()으로 커밋 처리해야한다.
    

#### select 결과 조회 메소드
- execute()로 실행한 SQL이 select인 경우 다음 메소드로 조회한다.
- fetchall()  : 조회된 모든 행을 한번에 가져올 때 사용한다. 결과를 tuple들을 묶은 리스트로 반환
- fetchone()  : 호출시 마다 한행씩 반환한다. PK로 조회한 경우 많이 사용한다.
- fetchmany(n): n행만큼 조회한다. n기본값-100. 특정개수반큼 반복문을 이용해 가져올때 사용.

In [15]:
import cx_Oracle 
from pprint import pprint # 자료구조를 보기 좋게 출력해주는 print함수 

# SQL문은 문자열로 작성. 
# 대소문자 구분안한다. 단, SQL의 키워드(SELECT, FROM, WHERE 등)은 보통 대문자로 , 컬럼명/테이블명은 소문자로 작성하는 것이 관례 
#  세미콜로(;)은 붙이지 않는다. 
# 주의 : 넣는 값 싱따옴표 구분해줘야함.why? 문자열이기 때문에  
select_sql = "SELECT emp_id, emp_name, job_id FROM emp"
insert_sql = "INSERT INTO emp VALUES (1500, '홍길동', 'FI_ACCOUNT', 101, to_date('2021/02/15','yyyy/mm/dd'), 30000, 0.23, 100)"  # 날짜형태 입력 시 to_date로 변환해서 
conn = None # Connection을 대입할 변수 
cursor = None

#INSERT 후에 SELECT 실행하기 -> commit 해줘야 db에 적용된다. 
try : 
    # 1. 연결 
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/xe')
    print(conn," : 연결완료")
    
    # 2. cursor생성 
    cursor = conn.cursor()
    
    # 3. sql실행 
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
    
    # 4. select 결과 조회
    result = cursor.fetchall() # 조회결과를 한번에 반환, 한행 - Tuple, 리스트로 튜플들을 묶어서 반환
    
    print(type(result), type(result[0])) # <class 'list'> <class 'tuple'>
    print(len(result))
    pprint(result)
    
    # 5. insert / update / delete 를 실행한 경우 commit 처리 **중요! 
    conn.commit()
    
except Exceptions as e : # 예외처리 
    print(e)
    
finally : 
    # 6. 연결 닫기 - connection, cursor 
    if cursor != None : 
        cursor.close()
    if conn != None :
        conn.close()

<cx_Oracle.Connection to c##scott_join@localhost:1521/xe>  : 연결완료
<class 'list'> <class 'tuple'>
108
[(100, 'Steven', 'AD_PRES'),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (116, 'Shelli', 'PU_CLERK'),
 (117, 'Sigal', 'PU_CLERK'),
 (118, 'Guy', 'PU_CLERK'),
 (119, 'Karen', 'PU_CLERK'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (125, 'Julia', 'ST_CLERK'),
 (126, 'Irene', 'ST_CLERK'),
 (127, 'James', 'ST_CLERK'),
 (128, 'Steven', 'ST_CLERK'),
 (129, 'Laura', 'ST_CLERK'),
 (130, 'Mozhe', None),
 (131, 'Jam

## placeholder(자리표시자)를 이용해 sql 실행
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식
    - 리스트나 튜플: placeholder 순서

In [16]:
import cx_Oracle
from pprint import pprint

In [32]:
select_sql = "SELECT * FROM emp WHERE salary BETWEEN :1 AND :2" # 값 변경할거니 자리 차리하고 있어라. placeholder 
insert_sql = "INSERT INTO emp VALUES (:id, :name, :job_id, :mgr_id, :to_date(:hire_date,'yyyymmdd'), :salary, :comm_pct, :dept_id)"

conn = None
cursor = None

try :
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/xe')
    cursor = conn.cursor()
    
    # SQL문 실행 
    # varchar2 / char / nvarchar / nchar - str 
    # number (정수/실수) - int / double 
    # date - datetime 모듈 , 지금은 to_date로 문자열을 변환하는 형태로 
    # null - None 
    param_dict = {
        'id' : 2000,
        'name' : '이순신', 
        'job_id' : 'FI_ACCOUNT', 
        'mgr_id' : 101, 
        'hire_date' : '20210213', 
        'salary' : 19990,
        'comm_pct' : None, 
        'dept_id' : 110
    }
    cursor.execute(insert_sql, param_dict)
    result0 = cursor.fetchall()
    pprint(result0)
    
#     cursor.execute(select_sql, [10000, 20000]) # 리스트나 튜플로 값을 알려준다. 
#     result1 = cursor.fetchall()
#     print("10000 ~ 20000 사이 조회")
#     pprint(result1)
    
#     cursor.execute(select_sql, [20000, 30000]) # 리스트나 튜플로 값을 알려준다. 
#     print("20000 ~ 30000 사이 조회")
#     result2 = cursor.fetchall()
#     pprint(result2)
    
    conn.commit()
     
except Exception as e :
    print(e)
    
finally: 
    if cursor != None :
        cursor.close()
    if conn != None : 
        conn.close()

ORA-01008: 일부 변수가 바인드되지 않았습니다.


## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.
- 반환타입 : tuple

### fetchall()
- 조회한 전체 결과셋을 반환한다.
- 한 row를 Tuple로 묶고 그 Tuple들을 List로 묶어서 반환한다.

### fetchmany(n)
- n 행씩 조회할 때 사용
- n 기본값 100

In [35]:
# emp_id(PK)로 직원정보를 조회하는 함수 
def select_emp_by_id(emp_id) :
#     conn, cursor = None, None 
    sql = "SELECT * FROM emp WHERE emp_id = :1"
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn : 
        with conn.cursor() as cursor : 
            # sql실행 
            cursor.execute(sql, [emp_id])
            result = cursor.fetchone() # 1행 조회(튜플로 반환). 조회결과가 없으면 None을 반환 
            return result 
        cursor.close()
    conn.close()

In [38]:
r = select_emp_by_id(100)
print(r)
print("id:",r[0], "이름:",r[1])

(100, 'Steven', 'AD_PRES', None, datetime.datetime(2003, 6, 17, 0, 0), 24000.0, None, 90)
id: 100 이름: Steven


In [39]:
r = select_emp_by_id(3000)
print(r)

None


In [44]:
r = select_emp_by_id(100)
r = select_emp_by_id(3000)
if r != None :
    print(r)
else :
    print('조회결과가 없습니다.', r)

조회결과가 없습니다. None


In [55]:
sql = 'SELECT * FROM emp order by emp_id'
with cx_Oracle.connect('c##scott/tiger@localhost:1521/XE') as conn : 
    with conn.cursor() as cursor : 
        cursor.execute(sql)
        result = cursor.fetchmany(10) # 10개만 조회
        print(len(result))
        print(result) # 100번 부터
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result) # 110번 부터 
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result) # 120번 부터
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result) # 130번 부터 
        
        # len(result)가 0이면 빠져나와라 형태로 반복문 돌리수 있음

10
[(100, 'Steven', 'AD_PRES', None, datetime.datetime(2003, 6, 17, 0, 0), 24000.0, None, 'Executive'), (101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 'Executive'), (102, 'Lex', 'AD_VP', 100, datetime.datetime(2001, 1, 13, 0, 0), 17000.0, None, 'Executive'), (103, 'Alexander', 'IT_PROG', 102, datetime.datetime(2006, 1, 3, 0, 0), 9000.0, None, 'IT'), (104, 'Bruce', 'IT_PROG', 103, datetime.datetime(2007, 5, 21, 0, 0), 6000.0, None, 'IT'), (105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 4800.0, None, 'IT'), (106, 'Valli', 'IT_PROG', 103, datetime.datetime(2006, 2, 5, 0, 0), 4800.0, None, 'IT'), (107, 'Diana', 'IT_PROG', 103, datetime.datetime(2007, 2, 7, 0, 0), 4200.0, None, 'IT'), (108, 'Nancy', 'FI_MGR', 101, datetime.datetime(2002, 8, 17, 0, 0), 12008.0, None, 'Finance'), (109, 'Daniel', 'FI_ACCOUNT', 108, datetime.datetime(2002, 8, 16, 0, 0), 9000.0, None, 'Finance')]
10
[(110, 'John', 'FI_ACCOUNT', 108, datetime.datetime(2005, 

### cursor를 for in 문을 이용해 select결과 조회
- select후 cursor는 Iterable 한 객체.
- cursor는 반복시마다 조회한 한행을 반환.

In [61]:
sql = "SELECT * from emp WHERE comm_pct IS NOT NULL ORDER BY emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn :
    with conn.cursor() as cursor :
        cursor.execute(sql)
        
        for idx, row in enumerate(cursor,start=1) : #한번 반복 할 때마다 cursor.fetchone()을 실행 
            print(f'{idx} :',row)
        

1 : (145, 'John', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 14000.0, 0.4, 80)
2 : (146, 'Karen', 'SA_MAN', 100, datetime.datetime(2004, 10, 1, 0, 0), 13500.0, 0.3, 80)
3 : (147, 'Alberto', 'SA_MAN', 100, datetime.datetime(2005, 3, 10, 0, 0), 12000.0, 0.3, 80)
4 : (148, 'Gerald', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 11000.0, 0.3, 80)
5 : (149, 'Eleni', 'SA_MAN', 100, datetime.datetime(2007, 10, 15, 0, 0), 10500.0, 0.2, 80)
6 : (150, 'Peter', 'SA_REP', 145, datetime.datetime(2007, 10, 15, 0, 0), 10000.0, 0.3, 80)
7 : (151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 9500.0, 0.25, 80)
8 : (152, 'Peter', 'SA_REP', 145, datetime.datetime(2005, 8, 20, 0, 0), 9000.0, 0.25, 80)
9 : (153, 'Christopher', 'SA_REP', 145, datetime.datetime(2006, 3, 30, 0, 0), 8000.0, 0.2, 80)
10 : (154, 'Nanette', 'SA_REP', 145, datetime.datetime(2006, 12, 9, 0, 0), 7500.0, 0.2, 80)
11 : (155, 'Oliver', 'SA_REP', 145, datetime.datetime(2007, 11, 23, 0, 0), 7000.0, 0.15,

In [62]:
sql = "SELECT job_id, sum(salary), round(avg(salary),2) FROM emp GROUP BY job_id"

with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn :
    with conn.cursor() as cursor : 
        cursor.execute(sql) 
        result = cursor.fetchall()
        pprint(result)

[('AD_VP', 34000, 17000),
 ('FI_ACCOUNT', 69600, 11600),
 ('PU_CLERK', 10800, 2700),
 ('SH_CLERK', 55900, 3288.24),
 ('HR_REP', 6500, 6500),
 ('PU_MAN', 20100, 10050),
 ('AC_MGR', 12008, 12008),
 ('ST_CLERK', 47900, 2817.65),
 ('AD_ASST', 4400, 4400),
 ('IT_PROG', 28800, 5760),
 (None, 16200, 2700),
 ('SA_MAN', 61000, 12200),
 ('AC_ACCOUNT', 8300, 8300),
 ('FI_MGR', 12008, 12008),
 ('ST_MAN', 36400, 7280),
 ('AD_PRES', 24000, 24000),
 ('MK_MAN', 13000, 13000),
 ('SA_REP', 250500, 8350),
 ('MK_REP', 6000, 6000),
 ('PR_REP', 10000, 10000)]


In [63]:
sql = "SELECT e.emp_name, d.dept_name FROM emp e left join dept d ON d.dept_id = d.dept_id"
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn :
    with conn.cursor() as cursor : 
        cursor.execute(sql) 
        result = cursor.fetchall()
        pprint(result)

[('Steven', '기획부'),
 ('Steven', '구매부'),
 ('Steven', 'Administration'),
 ('Steven', 'Marketing'),
 ('Steven', 'Purchasing'),
 ('Steven', 'Human Resources'),
 ('Steven', 'Shipping'),
 ('Steven', 'IT'),
 ('Steven', 'Public Relations'),
 ('Steven', 'Sales'),
 ('Steven', 'Executive'),
 ('Steven', 'Finance'),
 ('Steven', 'Accounting'),
 ('Steven', 'Treasury'),
 ('Steven', 'Corporate Tax'),
 ('Steven', 'Control And Credit'),
 ('Steven', 'Shareholder Services'),
 ('Steven', 'Benefits'),
 ('Steven', 'Manufacturing'),
 ('Steven', 'Construction'),
 ('Steven', 'Contracting'),
 ('Steven', 'Operations'),
 ('Steven', 'IT Support'),
 ('Steven', 'NOC'),
 ('Steven', 'IT Helpdesk'),
 ('Steven', 'Government Sales'),
 ('Steven', 'Retail Sales'),
 ('Steven', 'Recruiting'),
 ('Steven', 'Payroll'),
 ('Neena', '기획부'),
 ('Neena', '구매부'),
 ('Neena', 'Administration'),
 ('Neena', 'Marketing'),
 ('Neena', 'Purchasing'),
 ('Neena', 'Human Resources'),
 ('Neena', 'Shipping'),
 ('Neena', 'IT'),
 ('Neena', 'Public Rel

 ('Mattea', 'Shareholder Services'),
 ('Mattea', 'Benefits'),
 ('Mattea', 'Manufacturing'),
 ('Mattea', 'Construction'),
 ('Mattea', 'Contracting'),
 ('Mattea', 'Operations'),
 ('Mattea', 'IT Support'),
 ('Mattea', 'NOC'),
 ('Mattea', 'IT Helpdesk'),
 ('Mattea', 'Government Sales'),
 ('Mattea', 'Retail Sales'),
 ('Mattea', 'Recruiting'),
 ('Mattea', 'Payroll'),
 ('David', '기획부'),
 ('David', '구매부'),
 ('David', 'Administration'),
 ('David', 'Marketing'),
 ('David', 'Purchasing'),
 ('David', 'Human Resources'),
 ('David', 'Shipping'),
 ('David', 'IT'),
 ('David', 'Public Relations'),
 ('David', 'Sales'),
 ('David', 'Executive'),
 ('David', 'Finance'),
 ('David', 'Accounting'),
 ('David', 'Treasury'),
 ('David', 'Corporate Tax'),
 ('David', 'Control And Credit'),
 ('David', 'Shareholder Services'),
 ('David', 'Benefits'),
 ('David', 'Manufacturing'),
 ('David', 'Construction'),
 ('David', 'Contracting'),
 ('David', 'Operations'),
 ('David', 'IT Support'),
 ('David', 'NOC'),
 ('David', 'IT 

 ('Hermann', 'Treasury'),
 ('Hermann', 'Corporate Tax'),
 ('Hermann', 'Control And Credit'),
 ('Hermann', 'Shareholder Services'),
 ('Hermann', 'Benefits'),
 ('Hermann', 'Manufacturing'),
 ('Hermann', 'Construction'),
 ('Hermann', 'Contracting'),
 ('Hermann', 'Operations'),
 ('Hermann', 'IT Support'),
 ('Hermann', 'NOC'),
 ('Hermann', 'IT Helpdesk'),
 ('Hermann', 'Government Sales'),
 ('Hermann', 'Retail Sales'),
 ('Hermann', 'Recruiting'),
 ('Hermann', 'Payroll'),
 ('Shelley', '기획부'),
 ('Shelley', '구매부'),
 ('Shelley', 'Administration'),
 ('Shelley', 'Marketing'),
 ('Shelley', 'Purchasing'),
 ('Shelley', 'Human Resources'),
 ('Shelley', 'Shipping'),
 ('Shelley', 'IT'),
 ('Shelley', 'Public Relations'),
 ('Shelley', 'Sales'),
 ('Shelley', 'Executive'),
 ('Shelley', 'Finance'),
 ('Shelley', 'Accounting'),
 ('Shelley', 'Treasury'),
 ('Shelley', 'Corporate Tax'),
 ('Shelley', 'Control And Credit'),
 ('Shelley', 'Shareholder Services'),
 ('Shelley', 'Benefits'),
 ('Shelley', 'Manufacturing'

## INSERT / UPDATE/ DELETE
- SQL 실행 후 Connection객체의 commit()을 호출 해야 최종적으로 적용된다.
- commit()
    - 최종 적용
- rollback()
    - 변경전 상태로 되돌리기

### 사용할 테이블
```sql
create table test(
    id varchar2(10),
    name varchar2(100),
    age number(3));
```

## batch(일괄작업) 처리
- 한번에 메소드 호출로 다수 행을 처리한다.
- executemany(sql, placeholder에 전달할값)
    - placeholder에 전달할 값을 list로 묶어서 전달하면 sql문이 그 개수만큼 한번에 처리된다.

## TODO : 매개변수로 dept_id를 받아서 EMP 테이블에서 그 부서의 직원들의 salary를 두배 update하는 함수.

## TODO : 매개변수로 job_id를 받아서 그 업무를 하는 직원들을 EMP 테이블에서 삭제하는 함수.

# TODO : 매개변수로 직원 이름을 입력받아 EMP 테이블에서 그  직원들의 전체 정보를 조회결과를 출력하는 함수.

# TODO: 사용자로 부터 직원정보를 입력받아 EMP 테이블에 입력한다.
- emp_id, emp_name, job_id, mgr_id, hire_date, salary, comm_pct, dept_id 를 하나씩 입력받는다.
- job_id, mgr_id, salary, comm_pct, dept_id 을 입력하지 않은 경우 NULL 을 넣는다. (placehold 사용시 None을 대입)
- 입력된 값을 EMP 테이블에 insert 한다.
- 주의 job_id, mgr_id, dept_id는 FK 컬럼이므로 부모테이블의 PK값을 넣어야 한다.